In [ ]:
import pandas as pd
import random
import numpy as np

In [ ]:
# v4 is the 2 million dataset before cleaning, you can ask Li Cheng to share the csv with you.
# You may need to change the path. 
dat = pd.read_csv("/Users/komono/Desktop/400data_v4.csv",index_col = 0)
dat.head()

### Drop the columns Unnamed: 0.1 and upc(the unique item id)
### Convert binary_discount to 0 and 1

In [ ]:
dat.drop(columns = ["Unnamed: 0.1","upc"], axis = 1, inplace = True)

In [ ]:
dat['discount'] = np.where(dat['binary_discount'] == "discount_price", 1, 0)

In [ ]:
dat.drop(columns = ['binary_discount'], axis = 1, inplace = True)

In [ ]:
# The very original dataset is fairly balance, with seventy million rows, I random sampled into 2 million.
# Here v4 has 2 million rows.
dat['discount'].value_counts()

In [ ]:
dat.reset_index(drop = True, inplace = True)

In [ ]:
# You can update the path on your side.
# Save the current csv and do one-hot encoding later.
dat.to_csv("/Users/komono/Desktop/400data_v1.csv",index = False)

### Convert date to month and do One-Hot Encoding for month, zip, and state

In [ ]:
df = pd.read_csv("/Users/komono/Desktop/400data_v1.csv")

In [ ]:
from datetime import datetime

# Drop rows with null values in any of the three columns
df.dropna(subset = ['saledate','retail','orgprice'], inplace = True)

h = df['saledate'].astype(str).str.split("-")

month = [h[i][1] for i in range(len(df))]
df['month'] = month
df1 = pd.get_dummies(df, columns = ['month','zip','state'])
df1.head()

### Cluster brand

In [ ]:
import psycopg2
import matplotlib.pyplot as plt

In [ ]:
password = input("Password for MSiA: ")
conn = psycopg2.connect(
    host="pg.analytics.northwestern.edu",
    database="2022-everything-team08",
    user="zzp8676", #change here
    password=password)

print("Connected.")
cur = conn.cursor()

In [ ]:
import pandas.io.sql as psql

In [ ]:
query = """
SELECT brand, AVG(retail), count(*) 
FROM msia400.newskuinfo sku
JOIN msia400.skstinfo sk
ON sk.sku = sku.sku
GROUP BY brand
ORDER BY avg(retail) DESC
"""

df2 = psql.read_sql(query,conn)
df2['brand'] = df2['brand'].apply(lambda x: x.strip())
df2['avg_log'] = np.log(df2['avg']+1)
df2['count_log'] = np.log(df2['count'])

df2

In [ ]:
# Get all the brands.
all_brand = psql.read_sql('SELECT distinct brand FROM msia400.newskuinfo',conn)
all_brand['brand'] = all_brand['brand'].apply(lambda x: x.strip())

### The first cluster: No transaction occurred.

In [ ]:
# All brands in df2 have at least one transaction, so brands from all_brand that does not
# occur in df2['brands'] means no transaction, which is our first cluster.

empty_brand = []
for brand in all_brand['brand']:
    if brand.strip() not in list(df2['brand']):
        empty_brand.append(brand.strip())

print(len(empty_brand))
empty_brand = set(empty_brand)

In [ ]:
# get all brand that only have count = 1 in skst table. Brands with only one transaction. 
count1_brand = set(df2[df2['count_log'] == 0]['brand'])

In [ ]:
# split the rest into 4 clusters
n = df2[df2['count_log'] != 0].shape[0]//4
brand1 = set(df2[df2['count_log'] != 0].iloc[:n,:]['brand'])
brand2 = set(df2[df2['count_log'] != 0].iloc[n:2*n,:]['brand'])
brand3 = set(df2[df2['count_log'] != 0].iloc[2*n:3*n,:]['brand'])
brand4 = set(df2[df2['count_log'] != 0].iloc[3*n:,:]['brand'])

In [ ]:
# Two clusters defined as the two vertical lines at count_log = 0 and count_log = 1.
# The other four clusters were based on quartile. 
plt.scatter(df2['count_log'], df2['avg_log'])

In [ ]:
brand_cluster = []
df1['brand'] = df1['brand'].astype(str)

for brand in list(df1['brand']):
    
    b = brand.strip()
    
    if b in empty_brand:
        brand_cluster.append("Cluster_1")
    elif b in count1_brand:
        brand_cluster.append("Cluster_2")
    elif b in brand1:
        brand_cluster.append("Cluster_3")
    elif b in brand2:
        brand_cluster.append("Cluster_4")
    elif b in brand3:
        brand_cluster.append("Cluster_5")
    else:
        brand_cluster.append("Cluster_6")



In [ ]:
df1['brand_cluster'] = np.array(brand_cluster)

In [ ]:
df1['brand_cluster'].value_counts()

In [ ]:
df3 = pd.get_dummies(df1, columns = ['brand_cluster'])

In [ ]:
df3.drop(columns = ['saledate', 'label_y','orgprice','brand'], axis = 1, inplace = True)

In [ ]:
np.log(df3['retail'] + 1).hist(bins = 30)
df3['log_retail'] = np.log(df3['retail'] + 1)

In [ ]:
np.log(df3['cost'] + 1).hist(bins = 30)
df3['log_cost'] = np.log(df3['cost'] + 1)

In [ ]:
df3.drop(columns = ['retail','cost'],axis = 1, inplace = True)

In [ ]:
# This v2 csv is the dataset that I shared with you guys during last meeting. 
df3.to_csv("/Users/komono/Desktop/400data_v2.csv", index = False)